In [22]:
# System Libs
import multiprocessing as mp
import sys
import os
from glob import glob
from time import time
from pathlib import Path

# Other Libs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from sklearn.metrics import f1_score
from PIL import Image
from tqdm import tqdm

# Torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms import CenterCrop, Resize, ToTensor, Normalize
device = ("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Local Libs
from data_utils import MaskClassifierDataset
from models import BaseModel

cuda


In [23]:
# Path Setting
# Project Dir
dir_project = Path('/opt/ml')

# Code Dir
dir_code = dir_project.joinpath('code')

# Model Dir
dir_model = dir_project.joinpath('model')

# Code Data
dir_data = dir_project.joinpath('input/data')
dir_eval = dir_data.joinpath('eval')
dir_train = dir_data.joinpath('train')

# Train Data
dir_img = dir_train.joinpath('images')
file_train_raw = dir_train.joinpath('train.csv')
file_train = dir_train.joinpath('train_processed.csv')

In [24]:
# train_processed.csv 
train_df = pd.read_csv(file_train, index_col=0)
train_df.head(5)

,id,path,Mask,Age,Gender,Class
0,000001,/opt/ml/input/data/train/images/000001_female_...,Wear,>= 30 and < 60,Female,4
1,000001,/opt/ml/input/data/train/images/000001_female_...,Wear,>= 30 and < 60,Female,4
2,000001,/opt/ml/input/data/train/images/000001_female_...,Wear,>= 30 and < 60,Female,4
3,000001,/opt/ml/input/data/train/images/000001_female_...,Wear,>= 30 and < 60,Female,4
4,000001,/opt/ml/input/data/train/images/000001_female_...,Wear,>= 30 and < 60,Female,4


<br><br><br>
## 1. Dataset & DataLoader
---

In [4]:
preprocess = transforms.Compose(
    [
        transforms.ToTensor()
    ]
)

dataset = MaskClassifierDataset(train_df, transform=preprocess)
loader = DataLoader(dataset, shuffle=True, batch_size=10)

<br><br><br>
## 2. Model
---

In [2]:
# model = BaseModel(num_classes=18)
# model.to(device)

In [12]:
def resnet_finetune(model, classes):
	model = model()
	# class 18개로 분리
	model.fc = nn.Linear(in_features=512, out_features=classes, bias=True)
	
	print("네트워크 필요 입력 채널 개수", model.conv1.weight.shape[1])
	print("네트워크 출력 채널 개수 (예측 class type 개수)", model.fc.weight.shape[0])

	return model

In [14]:
from torchvision.models import resnet18

model = resnet_finetune(resnet18, 18).to(device)

네트워크 필요 입력 채널 개수 3
네트워크 출력 채널 개수 (예측 class type 개수) 18


<br><br><br>
## 3. Inference
---

In [6]:
# Hyper-parameter
learning_rate = 1e-4
epochs = 3

In [7]:
criterion = nn.MultiLabelSoftMarginLoss().to(device)
optimizer = optim.Adam(params=model.parameters(), lr=learning_rate)

In [20]:
for epoch in range(1, epochs+1):
    running_loss = 0.
    running_acc = 0.

    for idx, (images, labels) in enumerate(tqdm(loader)):
        images = torch.stack(list(images), dim=0).to(device)
        labels = torch.tensor(list(labels)).to(device)

        optimizer.zero_grad()

        with torch.set_grad_enabled(True):
            logits = model(images)
            _, preds = torch.max(logits, 1)
            loss = criterion(logits, labels)

            loss.backward()
            optimizer.step()

        running_loss += loss.item()*images.size(0)
        running_acc += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(loader.dataset)
    epoch_acc = running_acc / len(loader.dataset)

    print(f"현재 epoch-{epoch}의 데이터 셋에서 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}")
print("학습 종료!")

100%|██████████| 18900/18900 [05:41<00:00, 55.31it/s]


NameError: name 'phase' is not defined

In [21]:
    print(f"현재 epoch-{epoch}의 데이터 셋에서 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}")


현재 epoch-1의 데이터 셋에서 평균 Loss : 0.225, 평균 Accuracy : 0.004
